In [28]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv")

In [29]:
df=df.fillna(0)

In [30]:
df

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,0.0,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,0.0,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369
...,...,...,...,...,...,...,...,...,...,...,...
9699,140,5.0,164.0,2981.107371,17.3,2013,Europe,Diesel,Front-wheel drive,0.0,15.101802
9700,180,0.0,154.0,2439.525729,15.0,2004,USA,Gasoline,All-wheel drive,0.0,17.962326
9701,220,2.0,138.0,2583.471318,15.1,2008,USA,Diesel,All-wheel drive,-1.0,17.186587
9702,230,4.0,177.0,2905.527390,19.4,2011,USA,Diesel,Front-wheel drive,1.0,15.331551


In [31]:
df["origin"].unique()

array(['Europe', 'USA', 'Asia'], dtype=object)

In [32]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)


In [33]:
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [34]:
y_train=df_train.fuel_efficiency_mpg
y_val=df_val.fuel_efficiency_mpg
y_test=df_test.fuel_efficiency_mpg

In [44]:
del df_train["fuel_efficiency_mpg"]
del df_val["fuel_efficiency_mpg"]
del df_test["fuel_efficiency_mpg"]

In [45]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

In [46]:
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
X_val = dv.fit_transform(df_val.to_dict(orient='records'))
X_test = dv.fit_transform(df_test.to_dict(orient='records'))

In [47]:
from sklearn.tree import DecisionTreeRegressor, export_text
dr=DecisionTreeRegressor(random_state=1,max_depth=1)
dr.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1, random_state=1)

In [48]:
print(export_text(dr, feature_names=list(dv.get_feature_names_out())))

|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



In [49]:
from sklearn.ensemble import RandomForestRegressor

In [51]:
rf=RandomForestRegressor(n_estimators=10, random_state=1,n_jobs=-1)
rf.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [53]:
y_pred_val=rf.predict(X_val)

In [54]:
from sklearn.metrics import root_mean_squared_error

In [56]:
root_mean_squared_error(y_val,y_pred_val)

0.4599777557336148

In [68]:
array=[]
for x in range(1,21):
    array.append(x*10)
    

In [69]:
array

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200]

In [70]:
for y in array:
    rf=RandomForestRegressor(n_estimators=y, random_state=1,n_jobs=-1)
    rf.fit(X_train,y_train)
    y_pred_val=rf.predict(X_val)
    print(f"{y}, {round(root_mean_squared_error(y_val,y_pred_val),3)}")


10, 0.46
20, 0.454
30, 0.451
40, 0.448
50, 0.446
60, 0.445
70, 0.445
80, 0.445
90, 0.445
100, 0.444
110, 0.443
120, 0.444
130, 0.443
140, 0.443
150, 0.443
160, 0.443
170, 0.443
180, 0.442
190, 0.443
200, 0.443


In [73]:
for x in [10, 15, 20, 25]:
    avg=0
    for y in array:
       
        rf=RandomForestRegressor(n_estimators=y, random_state=1,max_depth=x,n_jobs=-1)
        rf.fit(X_train,y_train)
        y_pred_val=rf.predict(X_val)
        avg+=round(root_mean_squared_error(y_val,y_pred_val),3)
    print(f"{x},{y}, {round(avg/len(array),3)}")


10,200, 0.442
15,200, 0.445
20,200, 0.446
25,200, 0.446


In [74]:
rf=RandomForestRegressor(n_estimators=10, random_state=1,max_depth=20,n_jobs=-1)
rf.fit(X_train,y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [78]:
rf.feature_importances_

array([1.14707165e-02, 3.81809750e-04, 3.11842085e-04, 3.26932342e-03,
       3.43693411e-04, 3.36671988e-04, 1.60402148e-02, 3.18229841e-03,
       2.35867094e-03, 1.59113306e-03, 4.76103046e-04, 5.20358083e-04,
       5.55151959e-04, 9.59162013e-01])

In [77]:
dv

DictVectorizer(sparse=False)

In [79]:
dv.get_feature_names_out()

array(['acceleration', 'drivetrain=All-wheel drive',
       'drivetrain=Front-wheel drive', 'engine_displacement',
       'fuel_type=Diesel', 'fuel_type=Gasoline', 'horsepower',
       'model_year', 'num_cylinders', 'num_doors', 'origin=Asia',
       'origin=Europe', 'origin=USA', 'vehicle_weight'], dtype=object)

In [80]:
feat_names = dv.get_feature_names_out()
importances = rf.feature_importances_

df_imp = pd.DataFrame({
    "feature": feat_names,
    "importance": importances
})

df_imp.sort_values("importance", ascending=False, inplace=True)
print(df_imp)

                         feature  importance
13                vehicle_weight    0.959162
6                     horsepower    0.016040
0                   acceleration    0.011471
3            engine_displacement    0.003269
7                     model_year    0.003182
8                  num_cylinders    0.002359
9                      num_doors    0.001591
12                    origin=USA    0.000555
11                 origin=Europe    0.000520
10                   origin=Asia    0.000476
1     drivetrain=All-wheel drive    0.000382
4               fuel_type=Diesel    0.000344
5             fuel_type=Gasoline    0.000337
2   drivetrain=Front-wheel drive    0.000312


In [83]:
import xgboost as xgb
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [86]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [88]:
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred_val=rf.predict(dval)
print(f"{y}, {round(root_mean_squared_error(y_val,y_pred_val),3)}")

TypeError: float() argument must be a string or a real number, not 'DMatrix'